### [교점에 별 만들기](https://school.programmers.co.kr/learn/courses/30/lessons/87377)

- numpy 의 선형대수 방정식을 이용해 풀어보기

In [ ]:
import numpy as np
from itertools import combinations

# 선형대수를 이용한 교점 찾는 방정식
def find_inter(A, B):
    a1, b1, c1 = A
    a2, b2, c2 = B
    
    A = np.array([[a1, b1], [a2, b2]])
    B = np.array([-c1, -c2])
    
    # 두 직선이 평행
    if np.linalg.det(A) == 0: 
        return False, []

    inter_point = np.linalg.solve(A, B)
    return True, inter_point

def solution(line):
    points = []
    for (A, B) in combinations(line, 2):
        check, point = find_inter(A, B)
        if check and point[0].is_integer() and point[1].is_integer():
            points.append(point)
    
    if not points:
        return []
    elif len(points) == 1:
        return ['*']
    
    # scaling
    points.sort(key=lambda x: (-x[1], x[0]))
    min_y, max_y = points[-1][1], points[0][1]
    
    Xs = [x[0] for x in points]
    min_x, max_x = min(Xs), max(Xs)
    
    y_len = int(max_y - min_y) + 1
    x_len = int(max_x - min_x) + 1    
    scaled_points = [[int(x[0]-min_x), int(max_y-x[1])] for x in points]
    
    # scaling된 좌표를 기준으로 교점에 표시하기
    answer = [['.']*x_len for _ in range(y_len)]
    for (x, y) in scaled_points:
        answer[y][x] = '*'
    return ["".join(x) for x in answer]

-> numpy를 이용한 위 코드로 해결이 안되는데, 이유를 모르겠음

-> 조사 결과 numpy의 linalg를 사용할 경우, 해당 연산이 부동소수점으로 연산되면서 미세한 오차가 발생해 실제로는 정수값이어도, 4.000000001과 같은 값으로 반환되는 경우가 존재하기 때문이며, 따라서 아래 코드와 같이 미세한 차이는 인정하도록 처리해야함

In [ ]:
import numpy as np
from itertools import combinations

# 선형대수를 이용한 교점 찾는 방정식
def find_inter(A, B):
    a1, b1, c1 = A
    a2, b2, c2 = B
    
    A_matrix = np.array([[a1, b1], [a2, b2]])
    B_vector = np.array([-c1, -c2])
    
    # 두 직선이 평행
    if np.linalg.det(A_matrix) == 0:
        return False, []

    inter_point = np.linalg.solve(A_matrix, B_vector)
    return True, inter_point

def solution(line):
    points = []
    
    # 부동 소수점 오차를 허용하는 작은 값
    epsilon = 1e-6 # 1e-9와 같이 아주 작은 값으로 비교할 경우, 18번이 틀리며 96.6점이 됨

    for (A, B) in combinations(line, 2):
        check, point = find_inter(A, B)

        # 교점 좌표가 정수와 아주 가까운 값인지 확인
        if check:
            x_int = round(point[0])
            y_int = round(point[1])
            
            # 교점 좌표와 가장 가까운 정수 간의 차이가 허용 오차 이내인지 확인
            if abs(point[0] - x_int) < epsilon and abs(point[1] - y_int) < epsilon:
                points.append((x_int, y_int))

    if not points:
        return []
        
    # 중복 교점 제거 (e.g., 세 직선이 한 점에서 만나는 경우)
    points = sorted(list(set(points)))

    # 최소/최대 좌표 찾기
    min_x = min(p[0] for p in points)
    max_x = max(p[0] for p in points)
    min_y = min(p[1] for p in points)
    max_y = max(p[1] for p in points)
    
    width = max_x - min_x + 1
    height = max_y - min_y + 1
    
    # 그리드 생성 (얕은 복사 문제 해결)
    grid = [['.'] * width for _ in range(height)]
    
    # 좌표 변환 후 그리드에 별 표시
    for x, y in points:
        scaled_x = x - min_x
        scaled_y = max_y - y
        grid[scaled_y][scaled_x] = '*'
        
    return [''.join(row) for row in grid]

- 그냥 numpy 안쓰고 하는게 낫겠다...

In [7]:
from itertools import combinations

def solution(line):    
    def intersect(l1, l2):
        A1, B1, C1 = l1
        A2, B2, C2 = l2
        det = A1 * B2 - A2 * B1
        if det == 0:
            return None  # 두 선이 평행
        x = (B1 * C2 - B2 * C1) / det
        y = (A2 * C1 - A1 * C2) / det
        if x.is_integer() and y.is_integer():
            return (int(x), int(y))
        return None
    
    points = set()
    for (l1, l2) in combinations(line, 2):
        point = intersect(l1, l2)
        if point:
            points.add(point)
    
    if not points:
        return []

    xs, ys = zip(*points)
    min_x, max_x = min(xs), max(xs)
    min_y, max_y = min(ys), max(ys)
    
    width = max_x - min_x + 1
    height = max_y - min_y + 1
    
    grid = [['.'] * width for _ in range(height)]
    
    for (x, y) in points:
        grid[max_y - y][x - min_x] = '*'
    
    return [''.join(row) for row in grid]


In [8]:
line = [[1, -1, 0], [2, -1, 0], [4, -1, 0]]
solution(line)

['*']

### [불량 사용자](https://school.programmers.co.kr/learn/courses/30/lessons/64064)

In [ ]:
from itertools import product
from collections import defaultdict

def solution(user_id, banned_id):
    
    def is_match(user, banned):
        if len(user) != len(banned):
            return False
        for u, b in zip(user, banned):
            if b in ['*', u]:
                continue
            else:
                return False
        return True

    matches = []
    for ban in banned_id:
        targets = []
        for user in user_id:
            if is_match(user, ban):
                targets.append(user)
        matches.append(targets)

    answer = set()    
    for p in product(*matches):
        if len(set(p)) == len(banned_id):
            answer.add(tuple(sorted(p)))

    return len(answer)